In [52]:
import requests
from PIL import Image
import requests
import argparse
import csv
import base64
import sys
from io import BytesIO
from PIL import Image
import six
import shutil
import uuid
from os import path as pth
import os
import multiprocessing
import time

In [29]:
def scale(content, min_dim):


    image = Image.open(content)

    # no scaling, keep images full size
    if min_dim == -1:
        return image

    # aspect-ratio preserving scale so that the smallest dimension is `min_dim`
    width, height = image.size
    scale_dimension = width if width < height else height
    scale_ratio = float(min_dim) / scale_dimension

    if scale_ratio == 1:
        return image

    return image.resize(
        (int(width * scale_ratio), int(height * scale_ratio)),
        Image.ANTIALIAS,
    )


def read_image(response, min_dim=0):
    """ Download response in chunks and convert to a scaled Image object """

    content = BytesIO(response)
    content.seek(0)
    
    if min_dim > 0 :
        return scale(content, min_dim)
    else:
        return content


In [38]:
def get_data(text, save_dir, nb_images=10):
    try:
        r = requests.get("https://api.qwant.com/api/search/images",
            params={
                'count': nb_images,
                'q': text,
                't': 'images',
                'safesearch': 1,
                'locale': 'fr_FR',
                'uiv': 4
            },
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
            },
            stream=True, timeout=2.0
        )
    except Exception as e:
        print(e)
        return
    try:
        response = r.json().get('data').get('result').get('items')
    except Exception as e:
        print(e)
        return
    urls = [r.get('media') for r in response]
    for url in urls:
        id = str(uuid.uuid5(uuid.NAMESPACE_DNS, url)) + '.jpg'
        if 'base64,' in url:
            image_data = url.split(',')[1]
            buffered = BytesIO()
            im = Image.open(buffered, format="JPEG").convert("RGB")
            im.save(pth.join(save_dir,id))
        else:
            try:
                response = requests.get(url,stream=True, timeout=2.0)
            except Exception as e:
                print(e)
                continue
            if response.status_code < 400:
                try :
                    im = Image.open(read_image(response.content)).convert("RGB")
                    im.save(pth.join(save_dir,id))
                except Exception as e:
                    print(e)
    return

In [72]:
def consumer(query):
    while queue.empty():
        time.sleep(5.0)  # give the queue a chance to populate
    
    while not queue.empty():
        q = queue.get(block=True, timeout=None)
        save_dir = "/data/datasets/QwantCrawl/"+q
        if (not pth.isdir(save_dir)):
            os.mkdir(save_dir)
        get_data(q, save_dir)
        
        if queue.empty():
            time.sleep(0.1)
    print("Exited Consumer")

In [73]:
def producer(queries, queue):
    f = False
    for q in queries:
        print(q)
        queue.put(q, block=True, timeout=None)

In [ ]:
f = open("/data/datasets/log_images.csv")
reader = csv.reader(f)
queries = set()
for line in reader:
    if len(line) > 3:
        print(line)
    queries.add(line[1].replace('/', ' '))
print("Queries ", len(queries))
queue = multiprocessing.Queue(1000)
processes = [
    multiprocessing.Process(target=producer, args=(queries,queue))
]

for i in range(10):
    processes.append(multiprocessing.Process(target=consumer, args=[queue]))
print('There is :', len(processes), 'processes' )

for p in processes:
    p.start()

for p in processes:
    p.join()

Queries  3075042
There is : 11 processes

brooklyn nine nine logo
coucou amour gif
le tartare der echte franzose
&quot;louise bourgoin&quot; nue
dino basket quiver
alexander friedberg
signe égales
comment faire un encadré sur un titre sur word
lastenvelo
coupe 504
bilal hassani roi
joyeux anniversaire chat
puma suede black
silhouette panthère enfant coloriage
image du top 14
babysitter keeping  gif
oum
fraude fical
transport for london
père noël bourre
guilleme guirado photo
pollution des transport routier
herunterfahren
perçy jackson
most famous awards cinema
johnny depp wife
ark poster
voiture safari
pyrex lunch box
a330 mrtt istres
le pinguion batman
journaliste top gear france
ordinateur ports
edmond et ses amis
vache maquillee
amboise lucarne commons
zao wuki
bastoni arti marziali in due parti
barette a cheveau bois
image drawing two people sitting
ir c3525i
nespresso inissia ou pixie
gropius
crepe gif drole
&quot;american costumes&quot;
sjoberg
amel bent enfant
kinder bueno gif
c

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8580 bytes but only got 8372. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))


chef etoile photo famille
willie watson buster cruggs
porno sur instagram
pub macdo
chevaux neige
poop expansion
station de métro oullins
dennis quaid pictures
fauteuil   dessin
picasso  chaise canee
mom caught mast gifs
mclaren senna lego
HTTPSConnectionPool(host='www.decoderdigitale.net', port=443): Max retries exceeded with url: /wp-content/uploads/2010/03/i_soliti_ignoti.jpg (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
newsletter icon
jean françois copé
thermostatique radiateur
india brummitt
thylane blondeau parents
perizoma vero moda
lit superposé enfant
entete ordonnance vierge
petra v stockings
règlementation sur cuve gaz enterrée
alfa romeo leuchten
bracelet orbit
logo blackwiew
fiori carnivori
equtation voltige
images de tsunamie
lili mirojnick
cannot identify image file <_io.BytesIO object at 0x7eff043d7308>
maison plouigneau petite
charnière de capot defender
samuel elkana anne
jose

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/JpegImagePlugin.py:790: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn("Image appears to be a malformed MPO file, it will be "


HTTPSConnectionPool(host='i1.wp.com', port=443): Read timed out. (read timeout=2.0)
silverstone pwm 120 slim sst-fw121
veste billie
id cooling low profile cpu cooler
zx10r 2008 wallpaper
HTTPSConnectionPool(host='www.delavente.com', port=443): Max retries exceeded with url: /32677-thickbox_default/malette-multicouche-rangement-maquillage-et-accessoires.jpg (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
suche fröstelzwerge
meilleur oreillette bluetooth musique
casque de jeux vidéo avec micro
vaches race à viande
ashram de yoga sivananda
peavey mb1
HTTPSConnectionPool(host='www.delavente.com', port=443): Max retries exceeded with url: /32681-thickbox_default/malette-multicouche-rangement-maquillage-et-accessoires.jpg (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
carr geneva
la cene tableau
révolution cubaine
musée de l'érotisme berlin
787 9 dreamliner plan de la cabine
coupe de cheveux carré degradé
samsung eco bubble 7kg
frau drauss

...

xi jinping livre
swift giad
robot cube en bois
remission cancer uterus
gamestar
plque immatriculation reunion
HTTPSConnectionPool(host='www.kinedo.com', port=443): Max retries exceeded with url: /media/cache/thumbCol12/uploads/images/7f6087731c39db68682e3be5ff0ad625503666ad.jpg (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
jacques garcia jeune
famille singe
a350 images
image dessin fée
bateau tortue
margaret anne florence
carte joyeux anniversaire eurodisney
gneiss d’isua
brunette chubby
beziers carte
grand canyon arizona
dubstone.
les voeux entreprises les plus sympas
maison tournante
dessons animaux
von doesburg
HTTPSConnectionPool(host='imgs.steps.dragoart.com', port=443): Max retries exceeded with url: /how-to-draw-an-anime-tiger-step-6_1_000000098887_5.gif (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



eva green nackt
cartoon air plane
sculpture lalane
buche purée de marrons sucrée
commerce  moyen-âge
business modele canvas
HTTPSConnectionPool(host='drscdn.500px.org', port=443): Read timed out. (read timeout=2.0)
plage anthy
on di e a amazonia
photo reveil
agneau label rouge sisteron
vestige sparte
bernie wrightson
HTTPSConnectionPool(host='www.cityfarmer.fr', port=443): Max retries exceeded with url: /242-798-thickbox/cube-lumineux-exterieur.jpg (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
tatouage jérome bosch
crête carte europe
moji marienkäfer
fond d ecran ordi nature
mr bean face meme
bim cvc library
clara morgan casting anal
poule en délire
you are really dumb
HTTPSConnectionPool(host='www.kespel.de', port=443): Max retries exceeded with url: /images/88Orden_0607.jpg (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
entretien toyota
mona lisa smile
cannot identif

/opt/conda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag))


station stoneham
dessin tete de mort pirate
ryad saatouf
HTTPSConnectionPool(host='2.bp.blogspot.com', port=443): Read timed out. (read timeout=2.0)
www.pass-eduation.fr evaluation geographie sur lescregions,departements,comunes cm1 cm2
1649 autorpotraitpoussin
HTTPSConnectionPool(host='www.weekendnotes.com', port=443): Max retries exceeded with url: /im/005/04/balmoral-beach-nsw1.JPG (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out',)))
tome 15 catanzaro
HTTPSConnectionPool(host='www.automobiledimension.com', port=443): Max retries exceeded with url: /photos/interior/opel-vivaro-combi-ct-2015-boot.jpg (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",),))
perceval image
sils
entrelacs celtes ronds
symbole caraux
pute rousse
HTTPSConnectionPool(host='www.mesdemos.fr', port=443): Max retries exceeded with url: /wp-content/uploads/2018/02/commande-volets-roulants.jpg (Caused by P